# Statistical and Probabilistic Analysis of HDFS System Logs for Anomaly Detection


## 1- Dataset Import

In [4]:
%pip install kagglehub[pandas-datasets]

   ---------------------------------------- 0.0/11.0 MB ? eta -:--:--
    --------------------------------------- 0.3/11.0 MB ? eta -:--:--
   ----- ---------------------------------- 1.6/11.0 MB 4.4 MB/s eta 0:00:03
   --------- ------------------------------ 2.6/11.0 MB 4.6 MB/s eta 0:00:02
   -------------- ------------------------- 3.9/11.0 MB 4.8 MB/s eta 0:00:02
   ----------------- ---------------------- 4.7/11.0 MB 4.8 MB/s eta 0:00:02
   -------------------- ------------------- 5.5/11.0 MB 4.5 MB/s eta 0:00:02
   --------------------- ------------------ 6.0/11.0 MB 4.4 MB/s eta 0:00:02
   ------------------------ --------------- 6.8/11.0 MB 4.1 MB/s eta 0:00:02
   --------------------------- ------------ 7.6/11.0 MB 4.1 MB/s eta 0:00:01
   ------------------------------- -------- 8.7/11.0 MB 4.2 MB/s eta 0:00:01
   ---------------------------------- ----- 9.4/11.0 MB 4.2 MB/s eta 0:00:01
   -------------------------------------- - 10.5/11.0 MB 4.2 MB/s eta 0:00:01
   ---------

In [7]:
import os
import kagglehub
from kagglehub import KaggleDatasetAdapter

DATASET = "ayenuryrr/loghub-hdfs-hadoop-distributed-file-system-data"

# 1) Download once (local cache)
dataset_dir = kagglehub.dataset_download(DATASET)
print("Downloaded to:", dataset_dir)

# 2) Find only CSV files (relative paths)
csv_paths = []
for root, _, files in os.walk(dataset_dir):
    for f in files:
        if f.lower().endswith(".csv"):
            abs_path = os.path.join(root, f)
            rel_path = os.path.relpath(abs_path, dataset_dir)
            csv_paths.append(rel_path)

csv_paths = sorted(csv_paths)
print("CSV files found:")
for p in csv_paths:
    print(" -", p)

# 3) Load only CSVs
dfs = {}
for p in csv_paths:
    dfs[p] = kagglehub.load_dataset(
        KaggleDatasetAdapter.PANDAS,
        DATASET,
        p
    )

print("Loaded CSVs:", {k: v.shape for k, v in dfs.items()})


Resuming download from 1185939456 bytes (480328867 bytes left)...
Resuming download from https://www.kaggle.com/api/v1/datasets/download/ayenuryrr/loghub-hdfs-hadoop-distributed-file-system-data?dataset_version_number=3 (1185939456/1666268323) bytes left.


100%|██████████| 1.55G/1.55G [01:56<00:00, 4.12MB/s]

Extracting files...


Downloaded to: C:\Users\saied\.cache\kagglehub\datasets\ayenuryrr\loghub-hdfs-hadoop-distributed-file-system-data\versions\3
CSV files found:
 - HDFS_2k\HDFS_2k.log_structured.csv
 - HDFS_2k\HDFS_2k.log_templates.csv
 - HDFS_v1\preprocessed\Event_occurrence_matrix.csv
 - HDFS_v1\preprocessed\Event_traces.csv
 - HDFS_v1\preprocessed\HDFS.log_templates.csv
 - HDFS_v1\preprocessed\anomaly_label.csv
 - HDFS_v3_TraceBench\preprocessed\failure_trace.csv
 - HDFS_v3_TraceBench\preprocessed\normal_trace.csv
 - HDFS_v3_TraceBench\preprocessed\rowNumberResult.csv
 - HDFS_v3_TraceBench\tracebench\AN_Data_corruptBlk_r_00FDN_30C_1to19B_0to120INT_15RT_5WT\edge.csv
 - HDFS_v3_TraceBench\tracebench\AN_Data_corruptBlk_r_00FDN_30C_1to19B_0to120INT_15RT_5WT\event.csv
 - HDFS_v3_TraceBench\tracebench\AN_Data_corruptBlk_r_00FDN_30C_1to19B_0to120INT_15RT_5WT\operation.csv
 - HDFS_v3_TraceBench\tracebench\AN_Data_corruptBlk_r_00FDN_30C_1to19B_0to120INT_15RT_5WT\trace.csv
 - HDFS_v3_TraceBench\tracebench\AN_Da

C:\Users\saied\AppData\Local\Temp\ipykernel_24312\831157266.py:28: DeprecationWarning: Use dataset_load() instead of load_dataset(). load_dataset() will be removed in a future version.
  dfs[p] = kagglehub.load_dataset(


KaggleApiHTTPError: 404 Client Error.

Resource not found at URL: https://www.kaggle.com/datasets/ayenuryrr/loghub-hdfs-hadoop-distributed-file-system-data/versions/3
The server reported the following issues: Data not found
Please make sure you specified the correct resource identifiers.

## 2- Data Preprocessing